In [9]:
import sklearn
import sklearn.model_selection
import pickle
import numpy as np
import pandas as pd
from transformers import XLMTokenizer, RobertaModel
from tqdm import tqdm
from helpers import convert_to_onehot

In [10]:
df1 = pd.read_json('../data/slowosiec_data.json.gz', compression='gzip')
df1

,"('emotions',)","('markedness',)","('text',)","('lemma',)"
0,smutek,-1.0,Za swoje winy został skazany przez sąd na doży...,dożywocie
1,zlosc,-1.0,Za swoje winy został skazany przez sąd na doży...,dożywocie
2,strach,-1.0,Za swoje winy został skazany przez sąd na doży...,dożywocie
3,radosc,0.5,W moim czasie wolnym lubię odpoczywać na dział...,czas wolny
4,oczekiwanie,0.5,W moim czasie wolnym lubię odpoczywać na dział...,czas wolny
...,...,...,...,...
80406,smutek,-1.0,Po diabelsku potraktowałeś tego człowieka - ni...,po diabelsku
80407,zlosc,-1.0,Po diabelsku potraktowałeś tego człowieka - ni...,po diabelsku
80408,wstret,-1.0,Po diabelsku potraktowałeś tego człowieka - ni...,po diabelsku
80409,wstret,-0.5,"A zupka dzisiaj jakaś taka cieniuteńka, jakby ...",cieniuteńki


In [11]:
X = df1["('text',)"].to_numpy()
print(X[0])

Za swoje winy został skazany przez sąd na dożywocie bez możliwości wcześniejszego warunkowego zwolnienia. 


### Convert y to onehot

In [12]:
y = df1["('emotions',)"].to_numpy()
y[0]

'smutek'

In [13]:
y = np.array([convert_to_onehot(data) for data in y])
y[0]

array([0, 0, 0, 0, 1, 0, 0, 0, 0])

## Split dataset with the same label balance

In [14]:
splitter = sklearn.model_selection.StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=0)

train_X = None
test_X = None
train_y = None
test_y = None

for train_index, test_index in splitter.split(X, y):
    train_X, test_X = X[train_index], X[test_index]
    train_y, test_y = y[train_index], y[test_index]

## Get tokenizer and model

In [15]:
tokenizer = XLMTokenizer.from_pretrained("allegro/herbert-klej-cased-tokenizer-v1")
model = RobertaModel.from_pretrained("allegro/herbert-klej-cased-v1")

## Parse datasets into input vectors

In [16]:
def encode_text(text):
    encoded_input = tokenizer.encode(text, return_tensors='pt')
    outputs = model(encoded_input)
    response = outputs[1].detach().numpy()
    return response[0]

In [17]:
train_X_parsed = []
test_X_parsed = []
for x in tqdm(list(train_X)):
    train_X_parsed.append(encode_text(x))
    
for x in tqdm(list(test_X)):
    test_X_parsed.append(encode_text(x))

100%|██████████| 16066/16066 [17:53<00:00, 14.97it/s]


## Save parsed data to train and test datasets so i don't have to parse it again!!!

In [18]:
with open('../data/slowosiec_train.pickle', 'wb') as handle:
    pickle.dump((train_X_parsed, train_y), handle)
with open('../data/slowosiec_test.pickle', 'wb') as handle:
    pickle.dump((test_X_parsed, test_y), handle)

In [19]:
with open('../data/slowosiec_train_org.pickle', 'wb') as handle:
    pickle.dump(train_X, handle)
with open('../data/slowosiec_test_org.pickle', 'wb') as handle:
    pickle.dump(test_X, handle)